# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

In [1]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [2]:
corpus = parse_corpus('../data/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

In [5]:
query_keywords = {}
for qid, query in queries.items():
    keyword_set = process_text(query)
    query_keywords[qid] = {
        'text': query,
        'keywords': keyword_set
    }

query_keywords

{1: {'text': 'Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios',
  'keywords': {'académico',
   'de',
   'el',
   'en',
   'estudiantes',
   'impacto',
   'la',
   'los',
   'mental',
   'rendimiento',
   'salud',
   'universitarios'}},
 2: {'text': 'Actividades extracurriculares y bienestar emocional en el campus universitario',
  'keywords': {'actividades',
   'bienestar',
   'campus',
   'el',
   'emocional',
   'en',
   'extracurriculares',
   'universitario',
   'y'}},
 3: {'text': 'Estrategias universitarias para reducir el estrés en estudiantes',
  'keywords': {'el',
   'en',
   'estrategias',
   'estrés',
   'estudiantes',
   'para',
   'reducir',
   'universitarias'}}}

In [10]:
import math

def cosine_similarity(set1, set2):
    # Crear el conjunto de palabras únicas de ambos conjuntos
    words = list(set1.union(set2))
    # Crear los vectores de frecuencia para cada conjunto
    vec1 = [1 if word in set1 else 0 for word in words]
    vec2 = [1 if word in set2 else 0 for word in words]

    # Calcular el producto punto y las magnitudes
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(v ** 2 for v in vec1))
    magnitude2 = math.sqrt(sum(v ** 2 for v in vec2))
    
    # Evitar la división por cero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    return dot_product / (magnitude1 * magnitude2)

# Función para calcular la similitud de Jaccard entre dos conjuntos de palabras
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    # Evitar la división por cero
    if not union:
        return 0.0
    return len(intersection) / len(union)

def calculate_relevance(corpus, query_keywords):
    relevance = {}
    for qid, query_data in query_keywords.items():
        relevance[qid] = {}
        query_kw = query_data['keywords']
        for doc_id, doc_data in corpus.items():
            doc_kw = doc_data['keywords']

            cos_sim = cosine_similarity(query_kw, doc_kw)
            jac_sim = jaccard_similarity(query_kw, doc_kw)

            relevance[qid][doc_id] = {
                'cosine_similarity': cos_sim,
                'jaccard_similarity': jac_sim
            }

    return relevance

relevance = calculate_relevance(corpus, query_keywords)
relevance

{1: {1: {'cosine_similarity': 0.11785113019775793,
   'jaccard_similarity': 0.058823529411764705},
  2: {'cosine_similarity': 0.5892556509887896,
   'jaccard_similarity': 0.38461538461538464},
  3: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  4: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  5: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  6: {'cosine_similarity': 0.11785113019775793,
   'jaccard_similarity': 0.058823529411764705},
  7: {'cosine_similarity': 0.5892556509887896,
   'jaccard_similarity': 0.38461538461538464},
  8: {'cosine_similarity': 0.11785113019775793,
   'jaccard_similarity': 0.058823529411764705},
  9: {'cosine_similarity': 0.1091089451179962,
   'jaccard_similarity': 0.05555555555555555},
  10: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  11: {'cosine_similarity': 0.47140452079103173,
   'jaccard_similarity': 0.2857142857142857},
  12: {'cosine_similarity': 0.2581988897471611,
   'jaccard_similarity': 0.1333333333333333

In [13]:
def rank_results(relevance):
    rankings = {}
    for qid, scores in relevance.items():

        ranked_docs = sorted(scores.items(), key=lambda x: x[1]['cosine_similarity'], reverse=True)
        rankings[qid] = ranked_docs
    return rankings

ranked_documents = rank_results(relevance)

for qid, ranked_docs in ranked_documents.items():
    print(f"Query: {queries[qid]}")
    for rank, (doc_id, scores) in enumerate(ranked_docs, 1):
        print(f"Rank {rank}: Document {doc_id} ({corpus[doc_id]['title']})")

Query: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
Rank 1: Document 2 (Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.)
Rank 2: Document 7 (La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.)
Rank 3: Document 14 (Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.)
Rank 4: Document 13 (Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los grupos de estudio en el campus.)
Rank 5: Document 11 (Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.)
Rank 6: Document 19 (Factores de riesgo y estrategias para prevenir el agotamiento académico en estudiantes universitarios.)
Rank 7: Document 29 (Análisis de los hábitos de sueño y su relación con el rendimiento en exámenes universitarios.)
Rank 8: Docu